# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import pandas as pd
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col , monotonically_increasing_id
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql import types as t
from pyspark.sql import SparkSession

In [2]:
#def create_spark_session():
 #   spark = SparkSession \
  #      .builder \
   #     .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0","saurfang:spark-sas7bdat:2.0.0-s_2.11") \
    #    .enableHiveSupport().getOrCreate()
    #return spark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [35]:
# Read in the data here
#spark=create_spark_session()
df_airport = spark.read.format("csv").option("header", "true").load("airport-codes_csv.csv")
df_countryCodes=spark.read.format("csv").option("header", "true").load("i94cit_i94res.csv")
df_i94_mode=spark.read.format("csv").option("header", "true").load("i94mode.csv")
df_i94_visa=spark.read.format("csv").option("header", "true").load("i94visa.csv")
df_i94_add=spark.read.format("csv").option("header", "true").load("i94addr.csv")
df_i94_port=spark.read.format("csv").option("header", "true").load("i94port.csv")
#df_us_demographics=spark.read.option(Map "delimiter", ";" "header", "true").csv("us-cities-demographics.csv")
df_us_demographics=spark.read.format("csv").options(header="true",delimiter=";").load("us-cities-demographics.csv")

In [36]:
#Print Airport data schema
print(" Airport Data Schema...")
df_airport.printSchema()
# Print Airpot sample
print(" Airport Data sample..")
df_airport.head()
df_us_demographics.printSchema()
df_us_demographics.show(5)

 Airport Data Schema...
root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)

 Airport Data sample..
root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: string (nullable = true)
 |-- Male Population: string (nullable = true)
 |-- Female Population: string (nullable = true)
 |-- Total Population: string (nullable = true)
 |-- Number of Veterans: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Average Household Size: string (nullable = true)
 |-- State Code: string (nullable = true)
 |--

In [6]:
#Print Country data schema
print(" Country Data Schema...")

df_countryCodes.printSchema()
# Print Country sample
df_countryCodes.head()

#Print Airport data schema
print(" i94 mode Data Schema...")
df_i94_mode.printSchema()
# Print Airpot sample
print(" i94 Data sample..")
df_i94_mode.head()



 Country Data Schema...
root
 |-- code: string (nullable = true)
 |-- country: string (nullable = true)

 i94 mode Data Schema...
root
 |-- code: string (nullable = true)
 |-- transportation: string (nullable = true)

 i94 Data sample..


Row(code='1', transportation='Air')

In [7]:
df_airport.createOrReplaceTempView("airport_codes_df")
airport_codes_table=spark.sql(""" SELECT * FROM airport_codes_df """)
print("Sample Airport data....")
airport_codes_table.show(5)

df_countryCodes.createOrReplaceTempView ("countryCodes_df")
countryCodes=spark.sql (""" SELECT * FROM countryCodes_df""")
print("Sample Country Codes data....")
countryCodes.show(5)

df_i94_mode.createOrReplaceTempView("i94_mode_df")
i94_mode=spark.sql("""SELECT * FROM i94_mode_df """)

print("Sample 94 mode data....")
i94_mode.show(5)

df_i94_visa.createOrReplaceTempView("i94_visa_df")
i94_visa=spark.sql("""SELECT * FROM i94_visa_df""")
print("print Visa types.....")
i94_visa.show(5)

df_i94_add.createOrReplaceTempView("i94_add_df")
i94_state=spark.sql ("""SELECT * FROM i94_add_df""")
print (" print i94 state codes data....")
i94_state.show(5)

df_i94_port.createOrReplaceTempView("i94_port_df")
i94_port=spark.sql("""SELECT * FROM  i94_port_df""")
print(" Print port info...")
i94_port.show(5)




Sample Airport data....
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|     US-AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|   

In [8]:
	
#from pyspark.sql import SparkSession
#spark = SparkSession.builder.\
#config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
#.enableHiveSupport().getOrCreate()
#df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [9]:
df_spark.printSchema()
df_spark.createOrReplaceTempView("Immigration_df")

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [12]:
Immigration_df=spark.sql(""" SELECT * FROM Immigration_df """)

Immigration_df.show(5)                     

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1.897628485E9| null|      B2|
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SE

In [12]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

AnalysisException: 'path file:/home/workspace/sas_data already exists.;'

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [11]:
# Performing cleaning tasks here
#Check airport table for any nulls and drop the rows 
from pyspark.sql.functions import isnan, when, count, col

airport_codes_table.select([count(when(isnan(c), c)).alias(c) for c in airport_codes_table.columns]).show()

#Drop anyn null rows
airport_codes_table= airport_codes_table.na.drop()
airport_codes_table.show(5)
# Check the airport country before we use the data in our data model 
airport_codes_table.groupby('iso_country').count().show()

+-----+----+----+------------+---------+-----------+----------+------------+--------+---------+----------+-----------+
|ident|type|name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|coordinates|
+-----+----+----+------------+---------+-----------+----------+------------+--------+---------+----------+-----------+
|    0|   0|   0|           0|        0|          0|         0|           0|       0|        0|         0|          0|
+-----+----+----+------------+---------+-----------+----------+------------+--------+---------+----------+-----------+

+-----+-------------+--------------------+------------+---------+-----------+----------+-------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region| municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+--------

In [12]:
#since immigration data we are working is USA only , remove non US airport codes from airport dataset thus reduce the data size

airport_codes_table=airport_codes_table.filter(airport_codes_table.iso_country=='US')
airport_codes_table.show(5)

+-----+-------------+--------------------+------------+---------+-----------+----------+-------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region| municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+-------------+--------+---------+----------+--------------------+
| 07FA|small_airport|Ocean Reef Club A...|           8|       NA|         US|     US-FL|    Key Largo|    07FA|      OCA|      07FA|-80.274803161621,...|
| 0CO2|small_airport|Crested Butte Air...|        8980|       NA|         US|     US-CO|Crested Butte|    0CO2|      CSE|      0CO2|-106.928341, 38.8...|
| 0TE7|small_airport|   LBJ Ranch Airport|        1515|       NA|         US|     US-TX| Johnson City|    0TE7|      JCY|      0TE7|-98.6224975585999...|
| 13MA|small_airport|Metropolitan Airport|         418|       NA|         US

In [13]:
# Check diffrent airport types
airport_codes_table.groupby('type').count().show()

+--------------+-----+
|          type|count|
+--------------+-----+
| large_airport|  167|
| seaplane_base|   37|
|      heliport|   19|
|        closed|   18|
|medium_airport|  653|
| small_airport| 1015|
+--------------+-----+



In [13]:
# filter closed , heliport and seaplace base airport types since no immigration is collected in those places
non_airport=['closed','heliport','seaplane_base']
airport_codes_table=airport_codes_table.filter(~airport_codes_table.type.isin (non_airport))
airport_codes_table.groupby('type').count().show()

+--------------+-----+
|          type|count|
+--------------+-----+
| large_airport|  627|
|   balloonport|   24|
|medium_airport| 4550|
| small_airport|33965|
+--------------+-----+



In [14]:
#airport_codes_table['region']=airport_codes_table['iso_region'].str.strip().str.split("-", n = 1, expand = True)[1]
from pyspark.sql.functions import *
#airport_codes_table.show()
airport_codes_table = airport_codes_table.withColumn("iso_region", regexp_replace(airport_codes_table['iso_region'], 'US-', ''))
airport_codes_table.show(5)
#airport_codes_table['state'] = airport_codes_table['iso_region'].split("-", n = 1, expand = True)[1]


+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|        KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|        AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|       NA|         US|        AL|     Harvest|    00AL|     null|      00AL|-86.7703018188476...|
| 00AS|small_airport|      Fulton Airport|        1100|       NA|         US|     

In [21]:
# split coordinates columsn into Latitude and Longitude  columns
airport_codes_table=airport_codes_table.withColumn("Latitude",split(airport_codes_table['coordinates'],",").getItem(0))
airport_codes_table=airport_codes_table.withColumn("Longitude",split(airport_codes_table['coordinates'],",").getItem(1))
airport_codes_table.show(5)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+-------------------+-------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|           Latitude|          Longitude|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+-------------------+-------------------+
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|        KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|        -101.473911|          38.704022|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|        AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|     -151.695999146|        59.94919968|
| 00AL|small_airport|        Epps A

In [29]:
#Filter airport regions whcih are not in region (state) data set
#airport_df=airport_codes_table.join (i94_state,airport_codes_table.iso_region == i94_state.code,how='left')
#airport_df=airport_df.filter(airport_df['iso_region'].isNull())
#airport_df.show()
airport_codes_table.createOrReplaceTempView("airport_df")
airport_df=spark.sql(""" SELECT a.* FROM airport_df a  JOIN  i94_add_df b on a.iso_region =b.code """)
airport_df.show()

+-----+-------------+--------------------+------------+---------+-----------+----------+---------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|   municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+---------------+--------+---------+----------+--------------------+
| 07FA|small_airport|Ocean Reef Club A...|           8|       NA|         US|        FL|      Key Largo|    07FA|      OCA|      07FA|-80.274803161621,...|
| 0CO2|small_airport|Crested Butte Air...|        8980|       NA|         US|        CO|  Crested Butte|    0CO2|      CSE|      0CO2|-106.928341, 38.8...|
| 0TE7|small_airport|   LBJ Ranch Airport|        1515|       NA|         US|        TX|   Johnson City|    0TE7|      JCY|      0TE7|-98.6224975585999...|
| 13MA|small_airport|Metropolitan Airport|         418|       NA

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.